<a href="https://colab.research.google.com/github/pranavkantgaur/curves_and_surfaces/blob/master/curves_surfaces_monolithic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What are curves and surfaces?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import bezier  # Ensure you have the bezier library installed
from geomdl import BSpline

# Sample data points
x_data = np.array([0, 1, 2, 3])
y_data = np.array([0, 1, 4, 9])  # Quadratic function y = x^2

# Prepare a fine grid for plotting curves
x_fine = np.linspace(0, 3, 100)

# Linear Curve
y_linear = x_fine

# Quadratic Curve
y_quadratic = x_fine**2

# Cubic Curve
y_cubic = (x_fine**3) / 27  # Scale for better visualization

# Spline Curve
spline_tck = interpolate.splrep(x_data, y_data)
y_spline = interpolate.splev(x_fine, spline_tck)

# NURBS Curve (using geomdl)
nurbs_curve = BSpline.Curve()
nurbs_curve.degree = 3
nurbs_curve.ctrlpts = [[0, 0], [1, 1], [2, 4], [3, 1]]  # Control points for NURBS
nurbs_curve.knotvector = [0, 0, 0, 1, 1, 1]  # Example knot vector for a cubic NURBS
nurbs_points = nurbs_curve.evaluate(num_pts=100).pts
x_nurbs = nurbs_points[:, 0]
y_nurbs = nurbs_points[:, 1]

# Bézier Curve (using bezier library)
bezier_nodes = np.array([[0, 0], [1.5, 2.5], [2.5, 2], [3, 0]])  # Control points for Bézier curve
bezier_curve = bezier.Curve(bezier_nodes.T, degree=len(bezier_nodes) - 1)
bezier_points = bezier_curve.evaluate_list(np.linspace(0.0, 1.0, num=100)).T
x_bezier = bezier_points[:, 0]
y_bezier = bezier_points[:, 1]

# Plotting all curves
plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)
plt.plot(x_fine, y_linear, label='Linear')
plt.title('Linear Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 2)
plt.plot(x_fine, y_quadratic, label='Quadratic', color='orange')
plt.title('Quadratic Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 3)
plt.plot(x_fine, y_cubic, label='Cubic', color='green')
plt.title('Cubic Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 4)
plt.plot(x_fine, y_spline, label='Spline', color='red')
plt.title('Spline Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 5)
plt.plot(x_nurbs[:,0], y_nurbs[:,1], label='NURBS', color='purple')
plt.title('NURBS Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 6)
plt.plot(x_bezier, y_bezier,label='Bézier', color='brown')
plt.title('Bézier Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

# Adjust layout and show plot
plt.tight_layout()
plt.show()


## A running example
#### Developing ML model for nuclear cross-section estimation
* Scenario 1: The experimental dataset has cross-section values as a function of only energy. In this case, users can augment the dataset using various 1D curve generation methods.
* Scenario 2: The experimental dataset has cross-section values as a function of both energy and temperature. Here, users can augment the dataset using various 2D surface generation methods.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Function to perform 1D data augmentation using different curve generation methods
def augment_data_1d(energy_data, cross_section_data, method='spline'):
    energy_fine = np.linspace(min(energy_data), max(energy_data), num=100)

    if method == 'linear':
        augmented_cross_sections = np.interp(energy_fine, energy_data, cross_section_data)
    elif method == 'spline':
        spline_tck = interpolate.splrep(energy_data, cross_section_data)
        augmented_cross_sections = interpolate.splev(energy_fine, spline_tck)
    elif method == 'polynomial':
        coeffs = np.polyfit(energy_data, cross_section_data, deg=3)
        augmented_cross_sections = np.polyval(coeffs, energy_fine)
    else:
        raise ValueError("Unsupported curve generation method.")

    # Adding noise for augmentation
    noise = np.random.normal(0, 0.1 * augmented_cross_sections)  # 10% noise
    augmented_cross_sections += noise

    return energy_fine, augmented_cross_sections

# Function to perform 2D data augmentation using surface generation methods
def augment_data_2d(energy_data, temperature_data, cross_section_data, method='linear'):
    energy_fine = np.linspace(min(energy_data), max(energy_data), num=100)
    temperature_fine = np.linspace(min(temperature_data), max(temperature_data), num=100)

    # Create a meshgrid for 2D interpolation
    E_grid, T_grid = np.meshgrid(energy_fine, temperature_fine)

    if method == 'linear':
        # Linear interpolation on a grid
        Z = interpolate.griddata((energy_data, temperature_data), cross_section_data,
                                 (E_grid.ravel(), T_grid.ravel()), method='linear')
    elif method == 'cubic':
        # Cubic interpolation on a grid
        Z = interpolate.griddata((energy_data, temperature_data), cross_section_data,
                                 (E_grid.ravel(), T_grid.ravel()), method='cubic')
    else:
        raise ValueError("Unsupported surface generation method.")

    # Reshape back to grid shape
    Z = Z.reshape(E_grid.shape)

    return E_grid, T_grid, Z

# Load experimental cross-section dataset (for both scenarios)
def load_dataset(file_path):
    data = pd.read_csv(file_path)  # Assuming CSV format with appropriate columns
    return data

# Main function to run the training and evaluation process for both scenarios
def main():
    scenario = input("Select scenario (1 for Energy only, 2 for Energy and Temperature): ")

    if scenario == '1':
        # Load dataset for Scenario 1 (Energy only)
        data = load_dataset('cross_section_energy_only.csv')  # Replace with actual file path
        energy_data = data['Energy'].values
        cross_section_data = data['CrossSection'].values

        # Split into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            energy_data.reshape(-1, 1), cross_section_data, test_size=0.2, random_state=42)

        # Choose curve generation method for augmentation
        curve_method = input("Select curve generation method (linear/spline/polynomial): ")

        # Data augmentation on-the-fly during training
        augmented_energy, augmented_cross_sections = augment_data_1d(X_train.flatten(), y_train,
                                                                     method=curve_method)

        # Combine original and augmented data for training
        X_combined = np.vstack((X_train.flatten(), augmented_energy))
        y_combined = np.concatenate((y_train, augmented_cross_sections))

        # Train a neural network model
        model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000)
        model.fit(X_combined.reshape(-1, 1), y_combined)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Evaluate the model performance
        mse = mean_squared_error(y_test, y_pred)

        print(f"Mean Squared Error on Test Set: {mse:.4f}")

        # Plotting results for visualization
        plt.figure(figsize=(10, 6))
        plt.scatter(X_test, y_test, color='red', label='Test Data', zorder=5)
        plt.scatter(X_test, y_pred, color='blue', label='Predicted Data', zorder=4)
        plt.title('Neutron Cross-Section Prediction (Energy Only)')
        plt.xlabel('Energy (eV)')
        plt.ylabel('Cross Section (barns)')
        plt.legend()
        plt.grid()
        plt.show()

    elif scenario == '2':
        # Load dataset for Scenario 2 (Energy and Temperature)
        data = load_dataset('cross_section_energy_temperature.csv')  # Replace with actual file path
        energy_data = data['Energy'].values
        temperature_data = data['Temperature'].values
        cross_section_data = data['CrossSection'].values

        # Data augmentation using surface generation methods
        surface_method = input("Select surface generation method (linear/cubic): ")

        E_grid, T_grid, Z_augmented_cross_sections = augment_data_2d(
            energy_data, temperature_data, cross_section_data,
            method=surface_method)

        # Plotting the augmented surface data
        plt.figure(figsize=(12, 8))

        # Create a contour plot of the predicted cross-sections as a function of energy and temperature.
        contour_plot = plt.contourf(E_grid, T_grid, Z_augmented_cross_sections,
                                     levels=50, cmap='viridis')

        plt.colorbar(contour_plot)

        plt.title('Augmented Neutron Cross-Section Surface')
        plt.xlabel('Energy (eV)')
        plt.ylabel('Temperature (K)')

        plt.show()

if __name__ == "__main__":
    main()


## Lets dig deeper into the algorithms

### Bezier interpolation

In [ ]:
## Pseudocode

In [ ]:
## Failed test-cases

### Spline interpolation

In [ ]:
## Pseudocode

In [ ]:
## Failed test-cases

### B-spline interpolation

In [ ]:
## Pseudocode

In [ ]:
## Failed test-cases

### NURBS interpolation

In [ ]:
## Pseudocode

In [ ]:
## Failed test-cases

## Reflections

### Which one to use when?